In [1]:
query = """
SELECT * FROM (SELECT *\nFROM (\n  with reg_users as (\n  \n  \tselect * \n  \tfrom (\n  \tselect \n  \t        a.*\n  \t        , b.*\n  \t        , c.*\n  \t        , d.*\n  \tfrom \n  \t(\n  \tselect \n  \t        distinct \n  \t        anonymous_id\n  \t        , user_id\n  \tfrom mapbox_customer_data.segment_identifies\n  \twhere dt >= \'2018-07-01\'\n  \tand anonymous_id is not null\n  \tand user_id is not null\n  \t) a\n  \n  \tleft join \n  \n  \t(\n  \tselect \n  \t        id \n  \t        , email\n  \t        , created\n  \tfrom mapbox_customer_data.accounts\n  \twhere cast(dt as DATE) = CURRENT_DATE - INTERVAL \'1\' DAY \n  \t) b\n  \t        on a.user_id = b.id\n  \n  \tleft join \n  \n  \t(\n  \t    \n  \t        select        \n  \t                anonymous_id as anon_id_ad\n  \t                , context_campaign_name\n  \t                , min(timestamp) as min_exposure\n  \t        from mapbox_customer_data.segment_pages\n  \t        where dt >= \'2018-07-01\'\n  \t        and context_campaign_name is not null\n  \t        group by 1,2\n  \n  \t) c \n  \t        on a.anonymous_id = c.anon_id_ad\n  \t        \n  \tleft join \n  \n  \t(\n  \t        select \n  \t                distinct\n  \t                anonymous_id as anon_id_event\n  \t                , original_timestamp\n  \t                , event\n  \t                , context_traits_email\n  \t        from mapbox_customer_data.segment_tracks\n  \t        where dt >= \'2018-07-01\'\n  \t        and event like \'submitted_%form\'\n  \t        and context_traits_email is not null\n  \t) d\n  \t        on a.anonymous_id = d.anon_id_event\n  \n  \t) \n  \twhere context_campaign_name is not null\n  \n  ), \n  \n  non_reg_users as (\n  \n  \tselect \n  \t        context_campaign_name\n  \t        , min_exposure\n  \t        , event\n  \t        , original_timestamp as event_timestamp\n  \t        , context_traits_email as event_email\n  \tfrom (\n  \tselect a.*\n  \t        , b.*\n  \tfrom \n  \t(\n  \t        select \n  \t                anonymous_id as anon_id_ad\n  \t                , context_campaign_name\n  \t                , min(original_timestamp) as min_exposure\n  \t        from (       \n  \t        select        \n  \t                context_campaign_name\n  \t                , anonymous_id\n  \t                , original_timestamp \n  \t        from mapbox_customer_data.segment_pages\n  \t        where dt >= \'2018-07-01\'\n  \t        and context_campaign_name is not null\n  \t        )\n  \t        group by 1,2\n  \t) a\n  \n  \tleft join \n  \t(\n  \t        select \n  \t                distinct\n  \t                anonymous_id as anon_id_event\n  \t                , original_timestamp\n  \t                , event\n  \t                , context_traits_email\n  \t        from mapbox_customer_data.segment_tracks\n  \t        where dt >= \'2018-07-01\'\n  \t        and event like \'submitted_%form\'\n  \t        and context_traits_email is not null\n  \t) b\n  \t        on a.anon_id_ad = b.anon_id_event\n  \n  \t)\n  \twhere anon_id_event is not null\n  \tand to_unixtime(min_exposure) <= to_unixtime(original_timestamp)\n  \tand cast(min_exposure as DATE) >= cast(original_timestamp as DATE) - INTERVAL \'28\' DAY\n  \n  \n  ), \n  \n  mql_flag as (\n  \n  \tselect \n  \t        email\n  \t        , created_date\n  \t        , last_mql_date_c\n  \t        , mql_flag\n  \tfrom (\n  \n  \tselect \n  \t        email\n  \t        , min(created_date) created_date\n  \t        , max(last_mql_date_c) last_mql_date_c\n  \t        , case when max(last_mql_date_c) is not null then 1 else 0 end as mql_flag\n  \t        , sum(case when is_deleted = true then 1 else 0 end) as is_deleted\n  \tfrom sales.salesforce_leads\n  \twhere cast(dt as DATE) = CURRENT_DATE - INTERVAL \'1\' DAY \n  \tgroup by 1\n  \t)\n  \twhere mql_flag = 1\n  \tand is_deleted = 0\n  \n  ),\n  \n  cleaned_list as (\n  \n  \n  \tselect \n  \t\tdistinct \n  \t\t\t*\n  \tfrom (\n  \t\tselect \n  \t\t        context_campaign_name\n  \t\t        , min_exposure\n  \t\t        , \'created_an_account\' as event\n  \t\t        , created as event_timestamp\n  \t\t        , email as event_email\n  \t\tfrom reg_users        \n  \t\twhere to_unixtime(min_exposure) <= to_unixtime(created)\n  \t\tand cast(min_exposure as DATE) >= cast(created as DATE) - INTERVAL \'28\' DAY\n  \n  \n  \t\tunion all\n  \n  \n  \t\tselect \n  \t\t        context_campaign_name\n  \t\t        , min_exposure\n  \t\t        , event\n  \t\t        , original_timestamp as event_timestamp\n  \t\t        , context_traits_email as event_email\n  \t\tfrom reg_users      \n  \t\twhere to_unixtime(min_exposure) <= to_unixtime(original_timestamp)\n  \t\tand cast(min_exposure as DATE) >= cast(original_timestamp as DATE) - INTERVAL \'28\' DAY\n  \n  \t\tunion all\n  \n  \t\tselect * \n  \t\tfrom non_reg_users\n  \t)\n  \n  )\n  \n  \n  \n  select \n  \ta.*\n  \t, b.*\n  from cleaned_list a\n  left join mql_flag b \n  \ton a.event_email = b.email\n) "custom_sql_query"\nLIMIT 0) T LIMIT 0
"""

In [86]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re

In [3]:
formatter = column_parser.Parser(query)

In [4]:
formatted = formatter.format_query(query)
print(formatted)


SELECT *
FROM
  (SELECT *
   FROM
     (WITH reg_users AS
        (SELECT *
         FROM
           (SELECT a.* ,
                   b.* ,
                   c.* ,
                   d.*
            FROM
              (SELECT DISTINCT anonymous_id ,
                               user_id
               FROM mapbox_customer_data.segment_identifies
               WHERE dt >= '2018-07-01'
                 AND anonymous_id IS NOT NULL
                 AND user_id IS NOT NULL ) a
            LEFT JOIN
              (SELECT id ,
                      email ,
                      created
               FROM mapbox_customer_data.accounts
               WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON a.user_id = b.id
            LEFT JOIN
              (SELECT anonymous_id AS anon_id_ad ,
                      context_campaign_name ,
                      min(TIMESTAMP) AS min_exposure
               FROM mapbox_customer_data.segment_pages
               WHERE dt >= '2018-07-

In [5]:
cte_query = formatter.parse_cte(formatted)

In [7]:
cte_query.keys()

dict_keys(['(WITH reg_users', 'non_reg_users', 'mql_flag', 'main', 'cleaned_list'])

In [9]:
print(cte_query['main'])

SELECT context_campaign_name ,
                   min_exposure ,
                   'created_an_account' AS event ,
                   created AS event_timestamp ,
                   email AS event_email
            FROM reg_users
            WHERE to_unixtime(min_exposure) <= to_unixtime(created)
              AND cast(min_exposure AS DATE) >= cast(created AS DATE) - INTERVAL '28' DAY
            UNION ALL SELECT context_campaign_name ,
                             min_exposure ,
                             event ,
                             original_timestamp AS event_timestamp ,
                             context_traits_email AS event_email
            FROM reg_users
            WHERE to_unixtime(min_exposure) <= to_unixtime(original_timestamp)
              AND cast(min_exposure AS DATE) >= cast(original_timestamp AS DATE) - INTERVAL '28' DAY
            UNION ALL SELECT *
            FROM non_reg_users)) SELECT a.* ,
                                        b.*
      FROM clea

In [389]:
query_1 = """SELECT * FROM (WITH table AS (SELECT * FROM (SELECT COALESCE((a+b), 0) AS c FROM sfdc.accounts) SELECT * FROM table) AS custom_sql) AS sfdc_accounts 
LEFT JOIN (SELECT MAX(dt) FROM sfdc.cases) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt
WHERE dt > '2020-04-03' 
"""


In [390]:
query_2 = """SELECT *
FROM (
  SELECT u.name, b.customer_tier_c, b.name, m.account, b.x18_digit_account_id_c, s.id, m.platform, m.mobile_os, m.num_requests, Row_number() OVER(
    PARTITION BY s.id
  ) row_
  FROM wbr.map_requests_by_account m
  INNER JOIN (
    SELECT DISTINCT id
    FROM mapbox_customer_data.styles 
    WHERE cast(dt as DATE) >= CURRENT_DATE - INTERVAL '14' DAY
      AND sources LIKE '%mapbox-streets-v7%'
  ) s
  ON m.service_metadata_version = s.id
  LEFT JOIN (
    SELECT customer_tier_c, csm_c, name, mapbox_username_c, x18_digit_account_id_c
    FROM sfdc.accounts
    WHERE cast(dt as DATE) = CURRENT_DATE - INTERVAL '1' DAY
  ) b
  ON m.account = b.mapbox_username_c
  LEFT JOIN (
    SELECT name, id
    FROM sfdc.users
    WHERE cast(dt as DATE) = CURRENT_DATE - INTERVAL '1' DAY
  ) u
  ON b.csm_c = u.id
  WHERE cast(m.dt as DATE) >= CURRENT_DATE - INTERVAL '14' DAY
    AND m.service_metadata = 'custom'
    AND m.service = 'styles'
    AND b.customer_tier_c IN (
      'Tier 0',
      'Tier 1',
      'Tier 2',
      'Tier 3',
      'Tier 4'
    )
)
WHERE row_ = 1
AND m.service_metadata = 'custom'
ORDER BY 1, 4
LIMIT 5000
"""

In [401]:
query_3 = """SELECT * FROM (SELECT *\nFROM (\n  with reg_users as (\n  \n  \tselect * \n  \tfrom (\n  \tselect \n  \t        a.*\n  \t        , b.*\n  \t        , c.*\n  \t        , d.*\n  \tfrom \n  \t(\n  \tselect \n  \t        distinct \n  \t        anonymous_id\n  \t        , user_id\n  \tfrom mapbox_customer_data.segment_identifies\n  \twhere dt >= \'2018-07-01\'\n  \tand anonymous_id is not null\n  \tand user_id is not null\n  \t) a\n  \n  \tleft join \n  \n  \t(\n  \tselect \n  \t        id \n  \t        , email\n  \t        , created\n  \tfrom mapbox_customer_data.accounts\n  \twhere cast(dt as DATE) = CURRENT_DATE - INTERVAL \'1\' DAY \n  \t) b\n  \t        on a.user_id = b.id\n  \n  \tleft join \n  \n  \t(\n  \t    \n  \t        select        \n  \t                anonymous_id as anon_id_ad\n  \t                , context_campaign_name\n  \t                , min(timestamp) as min_exposure\n  \t        from mapbox_customer_data.segment_pages\n  \t        where dt >= \'2018-07-01\'\n  \t        and context_campaign_name is not null\n  \t        group by 1,2\n  \n  \t) c \n  \t        on a.anonymous_id = c.anon_id_ad\n  \t        \n  \tleft join \n  \n  \t(\n  \t        select \n  \t                distinct\n  \t                anonymous_id as anon_id_event\n  \t                , original_timestamp\n  \t                , event\n  \t                , context_traits_email\n  \t        from mapbox_customer_data.segment_tracks\n  \t        where dt >= \'2018-07-01\'\n  \t        and event like \'submitted_%form\'\n  \t        and context_traits_email is not null\n  \t) d\n  \t        on a.anonymous_id = d.anon_id_event\n  \n  \t) \n  \twhere context_campaign_name is not null\n  \n  ), \n  \n  non_reg_users as (\n  \n  \tselect \n  \t        context_campaign_name\n  \t        , min_exposure\n  \t        , event\n  \t        , original_timestamp as event_timestamp\n  \t        , context_traits_email as event_email\n  \tfrom (\n  \tselect a.*\n  \t        , b.*\n  \tfrom \n  \t(\n  \t        select \n  \t                anonymous_id as anon_id_ad\n  \t                , context_campaign_name\n  \t                , min(original_timestamp) as min_exposure\n  \t        from (       \n  \t        select        \n  \t                context_campaign_name\n  \t                , anonymous_id\n  \t                , original_timestamp \n  \t        from mapbox_customer_data.segment_pages\n  \t        where dt >= \'2018-07-01\'\n  \t        and context_campaign_name is not null\n  \t        )\n  \t        group by 1,2\n  \t) a\n  \n  \tleft join \n  \t(\n  \t        select \n  \t                distinct\n  \t                anonymous_id as anon_id_event\n  \t                , original_timestamp\n  \t                , event\n  \t                , context_traits_email\n  \t        from mapbox_customer_data.segment_tracks\n  \t        where dt >= \'2018-07-01\'\n  \t        and event like \'submitted_%form\'\n  \t        and context_traits_email is not null\n  \t) b\n  \t        on a.anon_id_ad = b.anon_id_event\n  \n  \t)\n  \twhere anon_id_event is not null\n  \tand to_unixtime(min_exposure) <= to_unixtime(original_timestamp)\n  \tand cast(min_exposure as DATE) >= cast(original_timestamp as DATE) - INTERVAL \'28\' DAY\n  \n  \n  ), \n  \n  mql_flag as (\n  \n  \tselect \n  \t        email\n  \t        , created_date\n  \t        , last_mql_date_c\n  \t        , mql_flag\n  \tfrom (\n  \n  \tselect \n  \t        email\n  \t        , min(created_date) created_date\n  \t        , max(last_mql_date_c) last_mql_date_c\n  \t        , case when max(last_mql_date_c) is not null then 1 else 0 end as mql_flag\n  \t        , sum(case when is_deleted = true then 1 else 0 end) as is_deleted\n  \tfrom sales.salesforce_leads\n  \twhere cast(dt as DATE) = CURRENT_DATE - INTERVAL \'1\' DAY \n  \tgroup by 1\n  \t)\n  \twhere mql_flag = 1\n  \tand is_deleted = 0\n  \n  ),\n  \n  cleaned_list as (\n  \n  \n  \tselect \n  \t\tdistinct \n  \t\t\t*\n  \tfrom (\n  \t\tselect \n  \t\t        context_campaign_name\n  \t\t        , min_exposure\n  \t\t        , \'created_an_account\' as event\n  \t\t        , created as event_timestamp\n  \t\t        , email as event_email\n  \t\tfrom reg_users        \n  \t\twhere to_unixtime(min_exposure) <= to_unixtime(created)\n  \t\tand cast(min_exposure as DATE) >= cast(created as DATE) - INTERVAL \'28\' DAY\n  \n  \n  \t\tunion all\n  \n  \n  \t\tselect \n  \t\t        context_campaign_name\n  \t\t        , min_exposure\n  \t\t        , event\n  \t\t        , original_timestamp as event_timestamp\n  \t\t        , context_traits_email as event_email\n  \t\tfrom reg_users      \n  \t\twhere to_unixtime(min_exposure) <= to_unixtime(original_timestamp)\n  \t\tand cast(min_exposure as DATE) >= cast(original_timestamp as DATE) - INTERVAL \'28\' DAY\n  \n  \t\tunion all\n  \n  \t\tselect * \n  \t\tfrom non_reg_users\n  \t)\n  \n  )\n  \n  \n  \n  select \n  \ta.*\n  \t, b.*\n  from cleaned_list a\n  left join mql_flag b \n  \ton a.event_email = b.email\n) "custom_sql_query"\nLIMIT 0) T LIMIT 0
"""

In [403]:
formatter = column_parser.Parser(query_3)
formatted = formatter.format_query(query_3)


In [408]:
query_list = formatted.split('\n')
query_list

['SELECT *',
 'FROM',
 '  (SELECT *',
 '   FROM',
 '     (WITH reg_users AS',
 '        (SELECT *',
 '         FROM',
 '           (SELECT a.* ,',
 '                   b.* ,',
 '                   c.* ,',
 '                   d.*',
 '            FROM',
 '              (SELECT DISTINCT anonymous_id ,',
 '                               user_id',
 '               FROM mapbox_customer_data.segment_identifies',
 "               WHERE dt >= '2018-07-01'",
 '                 AND anonymous_id IS NOT NULL',
 '                 AND user_id IS NOT NULL ) a',
 '            LEFT JOIN',
 '              (SELECT id ,',
 '                      email ,',
 '                      created',
 '               FROM mapbox_customer_data.accounts',
 "               WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON a.user_id = b.id",
 '            LEFT JOIN',
 '              (SELECT anonymous_id AS anon_id_ad ,',
 '                      context_campaign_name ,',
 '                      min(TIMESTAMP

In [409]:
def stitch_query(query_list):
    query = "\n".join(query_list)
    return query.lstrip(' \(').rstrip('\)')

In [410]:
stitch_query(sub_query)

"SELECT u.name,\n          b.customer_tier_c,\n          b.name,\n          m.account,\n          b.x18_digit_account_id_c,\n          s.id,\n          m.platform,\n          m.mobile_os,\n          m.num_requests,\n          Row_number() OVER(PARTITION BY s.id) row_\n   FROM wbr.map_requests_by_account m\n   INNER JOIN\n     (SELECT DISTINCT id\n      FROM mapbox_customer_data.styles\n      WHERE cast(dt AS DATE) >= CURRENT_DATE - INTERVAL '14' DAY\n        AND sources LIKE '%mapbox-streets-v7%' ) s ON m.service_metadata_version = s.id\n   LEFT JOIN\n     (SELECT customer_tier_c,\n             csm_c,\n             name,\n             mapbox_username_c,\n             x18_digit_account_id_c\n      FROM sfdc.accounts\n      WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON m.account = b.mapbox_username_c\n   LEFT JOIN\n     (SELECT name,\n             id\n      FROM sfdc.users\n      WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) u ON b.csm_c = u.id\n   WHERE ca

In [411]:
def delevel_query(query_list):
    line_level = []
    sub_query = []
    pos_delete, pos_where = len(query_list), len(query_list)
    
    for i, line in enumerate(query_list):
        if line.startswith('ORDER') or line.startswith('GROUP'):
            pos_delete = i
        if line.startswith('WHERE'):
            pos_where = i 
        if line.startswith('FROM'):
            pos_from = i-1
        if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN'):
            pos_join = i
  
    for line in query_list[:pos_from+2]:
        line_level.append((line, "level_1"))
        
    for line in query_list[pos_from+1:pos_where]:
        if line.startswith(' ') or line.startswith('FROM'):
            sub_query.append((line, "level_2"))
        else:
            line_level.append((line, "level_1"))

    for line in query_list[pos_where:pos_delete]:    
        line_level.append((line, "level_1"))

    return line_level, sub_query

In [412]:
query_list

['SELECT *',
 'FROM',
 '  (SELECT *',
 '   FROM',
 '     (WITH reg_users AS',
 '        (SELECT *',
 '         FROM',
 '           (SELECT a.* ,',
 '                   b.* ,',
 '                   c.* ,',
 '                   d.*',
 '            FROM',
 '              (SELECT DISTINCT anonymous_id ,',
 '                               user_id',
 '               FROM mapbox_customer_data.segment_identifies',
 "               WHERE dt >= '2018-07-01'",
 '                 AND anonymous_id IS NOT NULL',
 '                 AND user_id IS NOT NULL ) a',
 '            LEFT JOIN',
 '              (SELECT id ,',
 '                      email ,',
 '                      created',
 '               FROM mapbox_customer_data.accounts',
 "               WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON a.user_id = b.id",
 '            LEFT JOIN',
 '              (SELECT anonymous_id AS anon_id_ad ,',
 '                      context_campaign_name ,',
 '                      min(TIMESTAMP

In [413]:
delevel_query(query_list)[0]

[('SELECT *', 'level_1'), ('FROM', 'level_1'), ('LIMIT 0', 'level_1')]

In [414]:
sub_query_1 = delevel_query(query_list)[1]

In [415]:
sub_query_1

[('FROM', 'level_2'),
 ('  (SELECT *', 'level_2'),
 ('   FROM', 'level_2'),
 ('     (WITH reg_users AS', 'level_2'),
 ('        (SELECT *', 'level_2'),
 ('         FROM', 'level_2'),
 ('           (SELECT a.* ,', 'level_2'),
 ('                   b.* ,', 'level_2'),
 ('                   c.* ,', 'level_2'),
 ('                   d.*', 'level_2'),
 ('            FROM', 'level_2'),
 ('              (SELECT DISTINCT anonymous_id ,', 'level_2'),
 ('                               user_id', 'level_2'),
 ('               FROM mapbox_customer_data.segment_identifies', 'level_2'),
 ("               WHERE dt >= '2018-07-01'", 'level_2'),
 ('                 AND anonymous_id IS NOT NULL', 'level_2'),
 ('                 AND user_id IS NOT NULL ) a', 'level_2'),
 ('            LEFT JOIN', 'level_2'),
 ('              (SELECT id ,', 'level_2'),
 ('                      email ,', 'level_2'),
 ('                      created', 'level_2'),
 ('               FROM mapbox_customer_data.accounts', 'level_

In [272]:
formatter = column_parser.Parser(sub_query_1)
formatted = formatter.format_query(sub_query_1)
sub_query_1_list = formatted.split('\n')
sub_query_1_list

['SELECT u.name,',
 '       b.customer_tier_c,',
 '       b.name,',
 '       m.account,',
 '       b.x18_digit_account_id_c,',
 '       s.id,',
 '       m.platform,',
 '       m.mobile_os,',
 '       m.num_requests,',
 '       Row_number() OVER(PARTITION BY s.id) row_',
 'FROM wbr.map_requests_by_account m',
 'INNER JOIN',
 '  (SELECT DISTINCT id',
 '   FROM mapbox_customer_data.styles',
 "   WHERE cast(dt AS DATE) >= CURRENT_DATE - INTERVAL '14' DAY",
 "     AND sources LIKE '%mapbox-streets-v7%' ) s ON m.service_metadata_version = s.id",
 'LEFT JOIN',
 '  (SELECT customer_tier_c,',
 '          csm_c,',
 '          name,',
 '          mapbox_username_c,',
 '          x18_digit_account_id_c',
 '   FROM sfdc.accounts',
 "   WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON m.account = b.mapbox_username_c",
 'LEFT JOIN',
 '  (SELECT name,',
 '          id',
 '   FROM sfdc.users',
 "   WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) u ON b.csm_c = u.id",
 "WHERE c

In [416]:
query = "\n".join(sub_query_1_list)

In [417]:
query

"SELECT u.name,\n       b.customer_tier_c,\n       b.name,\n       m.account,\n       b.x18_digit_account_id_c,\n       s.id,\n       m.platform,\n       m.mobile_os,\n       m.num_requests,\n       Row_number() OVER(PARTITION BY s.id) row_\nFROM wbr.map_requests_by_account m\nINNER JOIN\n  (SELECT DISTINCT id\n   FROM mapbox_customer_data.styles\n   WHERE cast(dt AS DATE) >= CURRENT_DATE - INTERVAL '14' DAY\n     AND sources LIKE '%mapbox-streets-v7%' ) s ON m.service_metadata_version = s.id\nLEFT JOIN\n  (SELECT customer_tier_c,\n          csm_c,\n          name,\n          mapbox_username_c,\n          x18_digit_account_id_c\n   FROM sfdc.accounts\n   WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON m.account = b.mapbox_username_c\nLEFT JOIN\n  (SELECT name,\n          id\n   FROM sfdc.users\n   WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) u ON b.csm_c = u.id\nWHERE cast(m.dt AS DATE) >= CURRENT_DATE - INTERVAL '14' DAY\n  AND m.service_metadata = 'custo

In [279]:
sub_query_2 = delevel_query(sub_query_1_list)[0]

In [280]:
sub_query_2

[['SELECT u.name,',
  '       b.customer_tier_c,',
  '       b.name,',
  '       m.account,',
  '       b.x18_digit_account_id_c,',
  '       s.id,',
  '       m.platform,',
  '       m.mobile_os,',
  '       m.num_requests,',
  '       Row_number() OVER(PARTITION BY s.id) row_'],
 ('SELECT u.name,', 'level_1'),
 ('FROM wbr.map_requests_by_account m', 'level_1'),
 ('INNER JOIN', 'level_1'),
 ('LEFT JOIN', 'level_1'),
 ('LEFT JOIN', 'level_1'),
 ("WHERE cast(m.dt AS DATE) >= CURRENT_DATE - INTERVAL '14' DAY", 'level_1'),
 ("  AND m.service_metadata = 'custom'", 'level_1'),
 ("  AND m.service = 'styles'", 'level_1'),
 ("  AND b.customer_tier_c IN ('Tier 0',", 'level_1'),
 ("                            'Tier 1',", 'level_1'),
 ("                            'Tier 2',", 'level_1'),
 ("                            'Tier 3',", 'level_1'),
 ("                            'Tier 4')", 'level_1')]

In [182]:
def has_child():
    pass

In [ ]:
def assign_level(query_list):
    